<a href="https://colab.research.google.com/github/JulienDbrt/BGS-Invoice/blob/main/JSON_Brut_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install paddlepaddle-gpu paddleocr
!apt-get install -y poppler-utils
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
!sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install openai mistralai python-dotenv

In [ ]:
# Extraction simple du fichier JSON en CSV - sans la clé "Pages"

import glob
import json
import chardet
import csv
from json import JSONDecodeError

directory_path = '/content/drive/MyDrive/Complet'
output_file_path = '/content/json_final_output_json.csv'

def extract_data_json(json_file_path):
    """
    Extracts data from a JSON file and returns it in a dictionary format.
    """
    try:
        with open(json_file_path, 'rb') as file:
            raw_data = file.read()
            encoding = chardet.detect(raw_data)['encoding'] or 'utf-8'
            json_data = json.loads(raw_data.decode(encoding))
    except JSONDecodeError as e:
        print(f"Error decoding JSON file {json_file_path}: {str(e)}")
        return None

    data_mapped = {
        "vendorEmail": json_data.get("tiersEmail", ""),
        "uo_2": json_data.get("uo_2", ""),
        "invoiceNumber": json_data.get("numeroPiece", ""),
        "vendorCode": json_data.get("codeTiers", ""),
        "commitmentCode": json_data.get("codeEngagement", ""),
        "title": json_data.get("title", ""),
        "dueDate": json_data.get("dateEcheance", ""),
        "documentType": json_data.get("typeDocument", ""),
        "vendorSiret": json_data.get("tiersSiret", ""),
        "projectCode": json_data.get("codeProjet", ""),
        "ttc": json_data.get("ttc", ""),
        "tiers": json_data.get("tiers", ""),
        "ht": json_data.get("ht", ""),
        "dateDocument": json_data.get("dateDocument", ""),
        "subChapter": json_data.get("sousChapitre", ""),
        "commitment": json_data.get("engagement", ""),
        "project": json_data.get("projet", ""),
        "invoiceType": json_data.get("typeFacture", "")
    }

    return data_mapped

def process_json_files(directory_path, output_file_path):
    """
    Processes all JSON files in the given directory and writes the data into a CSV file.
    """
    json_files = glob.glob(f"{directory_path}/*.json")
    results = []

    for file_path in json_files:
        filename = file_path.split('/')[-1]
        invoice_data = extract_data_json(file_path)
        if invoice_data:
            invoice_data['filename'] = filename
            results.append(invoice_data)

    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            "filename", "vendorEmail", "uo_2", "invoiceNumber", "vendorCode", "commitmentCode", "title",
            "dueDate", "documentType", "vendorSiret", "projectCode", "ttc", "tiers", "ht", "dateDocument",
            "subChapter", "commitment", "project", "invoiceType"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in results:
            writer.writerow(item)

process_json_files(directory_path, output_file_path)


# Skipped :
64060
63257
63155
63820
63155


## Misltal AI

In [ ]:
# Extraction de "Pages", nettoyage de la données et envoie dans Mistral API

import os
import re
import csv
import glob
import json
import chardet
from tqdm import tqdm
from google.colab import userdata
from mistralai.client import MistralClient
from pydantic import BaseModel
from mistralai.models.chat_completion import ChatMessage

api_key = userdata.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

directory_path = '/content/drive/MyDrive/Complet'
output_file_path = '/content/json_final_output_mistral.csv'

class Result(BaseModel):
    vendorEmail: str
    uo_2: str
    invoiceNumber: str
    vendorCode: str
    commitmentCode: str
    title: str
    dueDate: str
    documentType: str
    vendorSiret: str
    projectCode: str
    ttc: str
    tiers: str
    ht: str
    dateDocument: str
    subChapter: str
    commitment: str
    project: str
    invoiceType: str

def estimate_token_count(text):
    """Estimate the number of tokens based on the text length.
    Rough estimation assuming an average token length, including spaces."""
    return len(text) / 5

def extract_data_json(json_file_path):
    """Extract and process data from a JSON file."""
    try:
        with open(json_file_path, 'rb') as file:
            raw_data = file.read()
            if not raw_data:
                print(f"Warning: The file {json_file_path} is empty.")
                return None
            encoding = chardet.detect(raw_data)['encoding'] or 'utf-8'
            json_data = json.loads(raw_data.decode(encoding))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from the file {json_file_path}: {e}")
        return None

    pages_data = json_data.get('pages', [])
    sentences = []

    for page in pages_data:
        if page.get('$type') == 'PageContent':
            for text_zone in page.get('Items', []):
                if text_zone.get('$type') == 'TextZone':
                    for line in text_zone.get('Ln', []):
                        for item in line.get('Items', []):
                            if item.get('$type') == 'Word':
                                sentences.append(item.get('Value', ''))

    sentence = ' '.join(sentences)
    file_name = os.path.basename(json_file_path)
    return {'sentence': sentence, 'file_name': file_name}

def process_json_files(directory_path, output_file_path):
    json_files = glob.glob(f"{directory_path}/*.json")
    results = []
    print(f"Processing {len(json_files)} files...")

    for file_path in tqdm(json_files, desc="Progress"):
        print(f"Processing file: {file_path}")
        invoice_data = extract_data_json(file_path)
        if invoice_data is None:
            continue

        sentence = invoice_data.get('sentence', '')
        token_count = estimate_token_count(sentence)
        if token_count > 32000:
            max_length = 32000 * 5
            invoice_data['sentence'] = sentence[:max_length]

        document_details = invoice_data

        token_count = estimate_token_count(document_details)
        print(f"Estimated token count for {file_path}: {token_count}")

        if token_count > 31000:
            print(f"{file_path} skipped due to token limit")
            continue

        model = 'mistral-small-latest'

        prompt = [
            ChatMessage(role="system", content='Create a JSON output with the following keys and their respective keys.  If the key is not present, the field is filled with an empty string or appears as NaN (Not a Number) in the dataset : vendor Email: Represents the email address of the vendor. uo_2: Denotes a specific organizational unit or department within a company, indicated by the key "uo_2". "numeroPiece": Contains the invoice number. "codeTiers": Corresponds to a unique code assigned to the vendor. "codeEngagement": Refers to a specific commitment or contract code associated with the invoice. "title": The title or name of the document, Title listed on the invoice, which usually describes the service or product sold. "dueDate": Indicates the due date of the invoice. "typeDocument": Specifies the type of document based on a pre-defined set of categories (09.01 - Factures / 09.04 - Factures développement (avant promesse) / 09.07 - Factures marketing / 09.08 - Factures travaux et prorata / 09.21 - Honoraires juridiques / 09.22 - Honoraires prescripteurs / 09.31 - Situations travaux MOE BPCC / 09.32 - Situations travaux MOE externe. »). "tiersSiret": A French SIRET number unique to each French company.  "codeProjet": Represents a unique code for the project associated with the invoice. "ttc": The total amount to be paid, including all taxes (TTC).  "tiers": The name of the third party or vendor.  "ht": The amount before taxes (HT). "dateDocument": The date the document. "sousChapitre": always 09 - FACTURES. "engagement": Describes the specific commitment or purpose associated with the invoice. "projet": The name or description of the project related to the invoice. "typeFacture": Indicates the type of the facture (facture or estimation de travaux).'),
            ChatMessage(role="user", content=f"Here are the document details: \n{document_details}, create a JSON output")
        ]

        chat_response = client.chat(model=model, response_format={"type": "json_object", "schema": Result.schema_json()}, messages=prompt)
        llm_output = chat_response.choices[0].message.content
        invoice_data["LLM_output"] = llm_output
        results.append(invoice_data)

    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
           "file_name", "LLM_output", "sentence"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in results:
            writer.writerow(item)

process_json_files(directory_path, output_file_path)

## OpenAI

In [ ]:
# Extraction de "Pages", nettoyage de la données et envoie dans OpenAI API (le modèle varie selon le nombre de tokens estimé)

import os
import re
import csv
import glob
import json
import openai
import chardet
from tqdm import tqdm
from google.colab import userdata
from pydantic import BaseModel

openai.api_key = userdata.get('OPENAI_API_KEY')

directory_path = '/content/drive/MyDrive/Complet'
output_file_path = '/content/json_final_output_openai.csv'

class Result(BaseModel):
    vendorEmail: str
    uo_2: str
    invoiceNumber: str
    vendorCode: str
    commitmentCode: str
    title: str
    dueDate: str
    documentType: str
    vendorSiret: str
    projectCode: str
    ttc: str
    tiers: str
    ht: str
    dateDocument: str
    subChapter: str
    commitment: str
    project: str
    invoiceType: str

def estimate_token_count(text):
    """Estimate the number of tokens based on the text length.
    Rough estimation assuming an average token length, including spaces."""
    return len(text) / 5

def extract_data_json(json_file_path):
    """Extract and process data from a JSON file."""
    try:
        with open(json_file_path, 'rb') as file:
            raw_data = file.read()
            if not raw_data:
                print(f"Warning: The file {json_file_path} is empty.")
                return None
            encoding = chardet.detect(raw_data)['encoding'] or 'utf-8'
            json_data = json.loads(raw_data.decode(encoding))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from the file {json_file_path}: {e}")
        return None

    pages_data = json_data.get('pages', [])
    sentences = []

    for page in pages_data:
        if page.get('$type') == 'PageContent':
            for text_zone in page.get('Items', []):
                if text_zone.get('$type') == 'TextZone':
                    for line in text_zone.get('Ln', []):
                        for item in line.get('Items', []):
                            if item.get('$type') == 'Word':
                                sentences.append(item.get('Value', ''))

    sentence = ' '.join(sentences)
    file_name = os.path.basename(json_file_path)
    return {'sentence': sentence, 'file_name': file_name}

def process_json_files(directory_path, output_file_path):
    json_files = glob.glob(f"{directory_path}/*.json")
    results = []
    print(f"Processing {len(json_files)} files...")

    for file_path in tqdm(json_files, desc="Progress"):
        print(f"Processing file: {file_path}")
        invoice_data = extract_data_json(file_path)
        if invoice_data is None:
            continue

        sentence = invoice_data.get('sentence', '')
        token_count = estimate_token_count(sentence)
        if token_count > 32000:
            max_length = 32000 * 5
            invoice_data['sentence'] = sentence[:max_length]

        document_details = invoice_data

        token_count = estimate_token_count(document_details)
        print(f"Estimated token count for {file_path}: {token_count}")

        model = 'gpt-4-0125-preview' if token_count > 15500 else 'gpt-3.5-turbo-0125'

        if token_count > 31000:
            print(f"{file_path} skipped due to token limit")
            continue

        prompt = [
            ChatMessage(role="system", content='Create a JSON output with the following keys and their respective keys.  If the key is not present, the field is filled with an empty string or appears as NaN (Not a Number) in the dataset : vendor Email: Represents the email address of the vendor. uo_2: Denotes a specific organizational unit or department within a company, indicated by the key "uo_2". "numeroPiece": Contains the invoice number. "codeTiers": Corresponds to a unique code assigned to the vendor. "codeEngagement": Refers to a specific commitment or contract code associated with the invoice. "title": The title or name of the document, Title listed on the invoice, which usually describes the service or product sold. "dueDate": Indicates the due date of the invoice. "typeDocument": Specifies the type of document based on a pre-defined set of categories (09.01 - Factures / 09.04 - Factures développement (avant promesse) / 09.07 - Factures marketing / 09.08 - Factures travaux et prorata / 09.21 - Honoraires juridiques / 09.22 - Honoraires prescripteurs / 09.31 - Situations travaux MOE BPCC / 09.32 - Situations travaux MOE externe. »). "tiersSiret": A French SIRET number unique to each French company.  "codeProjet": Represents a unique code for the project associated with the invoice. "ttc": The total amount to be paid, including all taxes (TTC).  "tiers": The name of the third party or vendor.  "ht": The amount before taxes (HT). "dateDocument": The date the document. "sousChapitre": always 09 - FACTURES. "engagement": Describes the specific commitment or purpose associated with the invoice. "projet": The name or description of the project related to the invoice. "typeFacture": Indicates the type of the facture (facture or estimation de travaux).'),
            ChatMessage(role="user", content=f"Here are the document details: \n{document_details}, create a JSON output")
        ]

        completion = client.chat.completions.create(model=model, response_format={ "type": "json_object" }, messages=prompt)
        llm_output = completion.choices[0].message.content
        invoice_data["LLM_output"] = llm_output
        results.append(invoice_data)

    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            "sentence", "file_name", "LLM_output"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in results:
            writer.writerow(item)

process_json_files(directory_path, output_file_path)
